In [1]:
import socket
import threading

In [2]:
from ecdsa import SigningKey, SECP256k1, VerifyingKey, BadSignatureError
import hashlib
import base64
import binascii
from datetime import datetime

class Wallet:
    """
        钱包
    """
    def __init__(self):
        """
            钱包初始化时基于椭圆曲线生成一个唯一的秘匙对，代表区块链上一个唯一的账户
        """
        self._private_key = SigningKey.generate(curve=SECP256k1)
        self._public_key = self._private_key.get_verifying_key()
    
    @property
    def address(self):
        """
            这里通过公匙生成地址
        """
        h = hashlib.sha256(self._public_key.to_pem())
        return base64.b64encode(h.digest())
    
    @property
    def pubkey(self):
        """
            返回公匙字符
        """
        return self._public_key.to_pem()
    
    def sign(self, message):
        """
            生成数字签名
        """
        h = hashlib.sha256(message.encode('utf8'))
        return binascii.hexlify(self._private_key.sign(h.digest()))

    
def verify_sign(pubkey, message, signature):
    """
        验证签名
    """
    verifier = VerifyingKey.from_pem(pubkey)
    h = hashlib.sha256(message.encode('utf8'))
    return verifier.verify(binascii.unhexlify(signature), h.digest())


import json
from json import JSONEncoder

class Transaction:
    """
        交易的结构
    """
    
    def __init__(self, sender, recipient, amount):
        """
            初始化交易，设置交易的发送方，接收方，和交易数量
        """
        if isinstance(sender, bytes):
            sender = sender.decode('utf-8')
        self.sender = sender
        if isinstance(recipient, bytes):
            recipient = recipient.decode('utf-8')
        self.recipient = recipient
        self.amount = amount
    
    def set_sign(self, signature, pubkey):
        """
            为了便于验证交易的可靠性，需要发送方输入他的公匙和签名
        """
        self.signature = signature
        self.pubkey = pubkey
    
    def __repr__(self):
        """
            交易大致可分为两种，一种是挖矿所得，二是转账交易
            挖矿物发送方，以此进行区分现实不同内容
        """
        if self.sender:
            s = "%s sending %d number of coins to %s"%(self.sender, self.amount, self.recipient)
        else:
            s = "%s getting %d number of coins from mining"%(self.recipient, self.amount)
        return s
    
    
    
class Block:
    """
        区块结构
            prev_hash:  父区块哈希值
            transactions:交易内容
            time stamp: 区块创建时间
            hash:       区块哈希值
            Nonce       随机数
    """
    
    def __init__(self, transactions, prev_hash):
        self.prev_hash = prev_hash
        self.transactions = transactions
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        #不计算区块的哈希值，设置初始Nonce和哈希为None
        self.nonce = None
        self.hash = None
        
    def __repr__(self):
        return"transactions: %s\n Hash:%s"%(self.transactions, self.hash)
    

    

class BlockChain:
    """
        区块链结构体
            blocks:  包含的区块列表
    """
    
    def __init__(self):
        self.blocks = []
    
    def add_block(self, block):
        """
        添加区块
        """
        self.blocks.append(block)


        
class TransactionEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__



class ProofOfWork:
    """
        工作量证明
    """
    
    def __init__(self, block, miner, difficult=5):
        self.block=block
        self.miner=miner
        
        #定义工作量难度，默认为5，表示有效的hash以5个0开头
        self.difficulty=difficult
        
        #添加工作奖励
        self.reward_amount=10
        
        
    def mine(self):
        """
            挖矿函数
        """
        i=0
        prefix='0'*self.difficulty
        
        #添加奖励
        t = Transaction(
            sender = "",
            recipient=self.miner.address,
            amount=self.reward_amount,
        )
        sig = self.miner.sign(json.dumps(t, indent=4, cls=TransactionEncoder))
        t.set_sign(sig, self.miner.pubkey)
        self.block.transactions.append(t)
        
        while True:
            message = hashlib.sha256()
            message.update(str(self.block.prev_hash).encode('utf-8'))
            message.update(str(self.block.transactions).encode('utf-8'))
            message.update(str(self.block.timestamp).encode('utf-8'))
            message.update(str(i).encode("utf-8"))
            digest=message.hexdigest()
            if digest.startswith(prefix):
                self.block.nonce=i
                self.block.hash=digest
                return self.block
            i+=1
        
    


def get_balance(user):
    balance = 0
    for block in blockchain.blocks:
        for t in block.transactions:
            if t.sender == user.address.decode():
                balance-=t.amount
            elif t.recipient == user.address.decode():
                balance+=t.amount
    return balance


In [3]:
import pickle

#定义一个全局列表保存所有结点
NODE_LIST = []

class Node(threading.Thread):
    
    def __init__(self, name, port, host="localhost"):
        threading.Thread.__init__(self, name=name)
        self.host = host          #服务器地址
        self.port = port          #每个结点对应一个唯一的端口
        self.name = name          #唯一的结点名称
        self.wallet = Wallet()    #唯一的钱包
        self.blockchain = None    #用来存储一个区块链副本
    
    
    def run(self):
        """
        结点运行
        """
        self.init_blockchain()
    
        #在指定端口进行监听
        sock = socket.socket( socket.AF_INET, socket.SOCK_STREAM )
        sock.bind(( self.host, self.port ) )
        NODE_LIST.append({
            "name":self.name,
            "host":self.host,
            "port":self.port
            })
        sock.listen(10)
        print(self.name, "running...")
        while True:
            connection, address=sock.accept()
            print(":)")
            try:
                print(self.name, "processing request...")
                self.handle_request(connection)
            except socket.timeout:
                print('too long! 超时')
            except Exception as e:
                print(e,)
            connection.close()
    
    def init_blockchain(self):
        """
            初始化当前结点的区块链
        """
        
        #若当时网络中已存在其他的结点，则从第一个结点获取区块链信息
        if NODE_LIST:
            host = NODE_LIST[0]['host']
            port = NODE_LIST[0]['port']
            name = NODE_LIST[0]['name']
            print( self.name, "sending initialization request to %s" %(name))
            sock = socket.socket( socket.AF_INET, socket.SOCK_STREAM )
            sock.connect((host, port))          #连接网络中的第一个结点
            sock.send(pickle.dumps('INIT'))     #发送初始化请求
            data = []
            while True:                         #读取区块链信息，直至完全获取后退出
                print("p1")
                buf = sock.recv(PER_BYTE)
                print("p1's PER_BYTE")
                print(PER_BYTE)
                if not buf:
                    break
                data.append(buf)
                if len(buf) < PER_BYTE:
                    break
            sock.close()                        #获取完成后关闭连接
            
            #将获取的区块链信息赋值到当前结点
            self.blockchain = pickle.loads( b''.join(data))
            print(self.name, "finish initialization")
        else:
            #如果是网络中的第一个结点，初始化一个创世区块
            block = Block(transactions=[],prev_hash="")
            w = ProofOfWork(block, self.wallet)
            genesis_block = w.mine()
            self.blockchain = BlockChain()
            self.blockchain.add_block(genesis_block)
            print("created genesis_block")
            
    
    def handle_request(self, connection):
        data=[]
        while True:   #不断读取请求数据直至读取完成
            print("p2")
            buf=connection.recv(PER_BYTE)
            print("p3")
            print("PER_BYTE equals to", PER_BYTE )
            if not buf:
                break
            data.append(buf)
        
            #若读取到的数据长度小于规定长度， 说明数据读取完成，退出
            if len(buf)<PER_BYTE:
                break
        t = pickle.loads(b''.join(data))
        if isinstance(t, Transaction):
            print("processing transaction request")
            if verify_sign(t.pubkey,
                          str(t),
                          t.signature):
                #验证交易签名没问题，生成一个新的区块
                print(self.name, "验证交易成功")
                new_block = Block(transaction=[t], prev_hash="")
                print(self.name, "Creating a new block...")
                w = ProofOfWork(new_block, self.wallet)
                block = w.mine()
                print( self.name, "Adding the new block to blockchain")
                self.blockchain.add_block(block)
                print( self.name, "Broadcasting the new block on internet...")
                self.broadcast_new_block(block)
            else:
                print(self.name, "Transaction Failed!")
        elif isinstance(t, Block):
            print("Processing the new block's request...")
            if self.verify_block(t):
                print(self.name, "Block verified successfully")
                self.blockchain.add_block(t)
                print(self.name, "Block Added successfully")
            else:
                print(self.name, "Block verified failed")
        else:#如果不是新区块消息，默认为初始化消息类型，返回本地区块链内容
            connection.send(pickle.dumps(self.blockchain))
                
    
    def broadcast_new_block(self, block):
        """
            将新生成的区块广播到网络中其他结点
        """
        for node in NODE_LIST:
            host=node['host']
            port=node['port']
            
            if host ==self.host and port ==self.port:
                print(self.name, "Skipping my own node")
            else:
                print(self.name, "Broadcasting new block to %s" %(node['name']))
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.connect((host, port)) #连接到网络中的结点
                sock.send(pickle.dumps(block))  #发送新区块
                sock.close()   #发送完成后关闭连接
    
    def submit_transaction(self, transaction):
        for node in NODE_LIST:
            host=node['host']
            port=node['port']
            
            if host==self.host and port==sel.port:
                print(self.name, "Skipping my own node")
            else:
                print(self.name, "Broadcasting new block to $s:%s" %(self.host, self.port))
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.connect((node["host"], node["port"]))
                sock.send(pickle.dumps(transaction))  
                sock.close()  
                
    def print_blockchain(self):
        print("There are %d number of blocks in it\n"%len(self.blockchain.blocks))

        """
        for block in self.blockchain.blocks:
            print("previous hash: %s"%block.prev_hash)
            print("transaction: %s"%block.transactions)
            print("time stamp: %s"%block.timestamp)
            print("hash %s\n"%block.hash)
        """

In [4]:
def print_blockchain(Node):
    print("There are %d number of blocks in it\n"%len(Node.blockchain.blocks))
    """
    for block in Node.blockchain.blocks:
            print("previous hash: %s"%block.prev_hash)
            print("transaction: %s"%block.transactions)
            print("time stamp: %s"%block.timestamp)
            print("hash %s\n"%block.hash)
    """

In [5]:
#初始化结点1

node1 = Node(name="结点1", port=6008)

print(node1.host)

localhost


In [6]:
node1.start()

created genesis_block
结点1 running...


In [7]:
print(len(node1.blockchain.blocks))
for x in node1.blockchain.blocks:
    print(x.timestamp)

1
2022-08-16 10:57:21


In [8]:
node2 = Node("结点2", 6009)

In [9]:
node2.start()

结点2 sending initialization request to 结点1
:)p1


Exception in thread 结点2:
Traceback (most recent call last):
  File "/koko/system/anaconda/envs/python39/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "<ipython-input-3-2e6a8d6ec1d6>", line 21, in run
  File "<ipython-input-3-2e6a8d6ec1d6>", line 62, in init_blockchain
NameError: name 'PER_BYTE' is not defined



结点1 processing request...
p2
name 'PER_BYTE' is not defined


In [126]:
node2.print_blockchain()

AttributeError: 'NoneType' object has no attribute 'blocks'

In [127]:
print(len(node1.blockchain.blocks))
for x in node1.blockchain.blocks:
    print(x.timestamp)

1
2022-08-09 17:24:31
